In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn
import statsmodels.api as sm
from scipy.stats import zscore
from statsmodels import regression
from statsmodels.tsa.stattools import coint

In [51]:
# import MetaTrader5 as mt5
from datetime import datetime
import pytz
import time
from IPython.display import clear_output

In [4]:
class Pair_trading():
    def __init__(self, data_var,data_preco, par, valor_inicial):
        self.financeiro = valor_inicial
        self.posicionado = 0 #comeca sem nenhuma operação aberta
        self.lista_resultado_operacoes = []
        self.lista_evolucao_carteira = []
        self.lista_dias_carregando = []
        self.lista_rentabilidade_operacao = []
        self.contador_dias = 0 
        self.perm_pct_acumulado = 1
        #guarda tickers das acoes
        self.t1 = par[0] 
        self.t2 = par[1]
        self.volume_operacao = 0

        self.dados_var = data_var
        self.dados_preco = data_preco

    def calcula_zscore_spread(self,stock1,stock2):
        stock1 = sm.add_constant(stock1)
        results = sm.OLS(stock2,stock1).fit()
        stock1 = stock1[self.t1]
        b = results.params[self.t1]
        spread = stock2 - (b*stock1)
        zs = zscore(spread)[-1]
        return zs
        
    def condicoes_operar(self,zs, gatilho_high, preco_s1, preco_s2, index):
        if zs >= gatilho_high and self.posicionado == 0:
            #parametros para iniciar operacao
            self.posicionado = 1 #entrou em uma operacao            
            self.volume = (self.financeiro/2)//preco_s1
            self.razao = preco_s1/preco_s2
            self.contador_dias = index
            #guarda dados de quantas ações está comprado/vendido
            self.qtd_s1 =  self.volume
            self.qtd_s2 =  self.volume*self.razao
            #calcula valor total da compra e venda (idealmente devem ser iguais)
            valor_operado_s1 =  preco_s1*self.qtd_s1 #vende  s1 (credita caixa)
            valor_operado_s2 = -preco_s2*self.qtd_s2 #compra s2 (debita caixa) 
#             print(f'v1  R${valor_operado_s1:.2f} | v2 R${valor_operado_s2:.2f}')
#             print(f'C1  R${preco_s1:.2f} | D2 R${(preco_s2*self.razao):.2f}')
            self.financeiro += valor_operado_s1 + valor_operado_s2

        elif zs <= 0.08 and self.posicionado == 1:
            #calcula preço das açoes que estão em carteira
            valor_operado_s1 = -self.qtd_s1*preco_s1 #compra s1 (debita  caixa) 
            valor_operado_s2 =  self.qtd_s2*preco_s2 #vende  s2 (credita caixa)
#             print(f'v1 R${valor_operado_s1:.2f} | v2  R${valor_operado_s2:.2f}')
#             print(f'D1  R${preco_s1:.2f} | C2 R${(preco_s2*self.razao):.2f}')
            resultado_trade = valor_operado_s1 + valor_operado_s2 
#             print(f'resu: R${resultado_trade:.2f}\n')
            self.financeiro += resultado_trade 
            #zera quantidades de ações sendo operadas
            self.qtd_s1 -= self.qtd_s1
            self.qtd_s2 -= self.qtd_s2
            #guarda dados para acompanhamento ao longo do tempo
            pct_change_trade = resultado_trade/(self.financeiro-resultado_trade)
            self.perm_pct_acumulado = self.perm_pct_acumulado*(1+pct_change_trade)
            self.lista_rentabilidade_operacao.append(pct_change_trade*100)
            self.lista_evolucao_carteira.append(self.financeiro)
            self.lista_dias_carregando.append(index-self.contador_dias)
            self.lista_resultado_operacoes.append(resultado_trade)
            #zera parametros para próximas operações 
            self.posicionado = 0
            self.razao = 0
            self.volume = 0
            self.contador_dias = 0
        
    def opera(self):
        period = 45
        c = 0
        s1 = self.dados_var[self.t1]
        s2 = self.dados_var[self.t2]
        for index in range(period, len(self.dados_var)):
            j1 = (s1[c:index]) #seleciona a acao 1 com uma janela com *period* dias 
            j2 = (s2[c:index])

            preco_s1 = self.dados_preco[self.t1][self.dados_var.index == index].values[0]
            preco_s2 = self.dados_preco[self.t2][self.dados_var.index == index].values[0]
            
            zs = self.calcula_zscore_spread(stock1=j1,stock2=j2)
            self.condicoes_operar(zs=zs, gatilho_high = 1.1, preco_s1=preco_s1, preco_s2=preco_s2, index=index)
            c += 1

        

In [23]:
data = pd.read_excel('/home/daniel/Desktop/Projetos/mql_python/Relatório 1/dados_relatorio.xlsx')[:]
data = data[['time','close FLRY3','close USIM5']]
data_var = data.drop('time',axis=1).pct_change()[-44:]
data_preco = data[-44:]

In [24]:
index = np.arange(0,len(data_var),1)

In [25]:
data_var=data_var.set_index(index)
data_preco= data_preco.set_index(index)

In [26]:
data_var


,close FLRY3,close USIM5
0,-0.006642,-0.031039
1,0.016298,-0.002786
2,0.020148,0.004190
3,0.001612,0.015299
4,0.003219,0.000000
5,-0.003209,-0.015068
6,0.002817,0.022253
7,-0.000803,0.023129
8,0.007631,0.000000
9,0.005978,0.013298


In [44]:
tickers = [['close BBDC3','close BTOW3'],['close ECOR3','close GOLL4'],['close CSNA3','close MRFG3'],['close QUAL3','close USIM5']] #['close VALE3','close ITSA4'],['close BBAS3','close BBSE3'],['close BBDC4','close ITUB4']

In [50]:
#quando for dar o input dos dados, colocar 1 row a mais
timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2020, 8, 4, tzinfo=timezone) #ano/mes/dia

for par_usado in (tickers):
    #cria dataset com apenas os fechamentos do par
    data_par = data[par_usado]
    ts1,ts2 = (i[0][6:]),(i[1][6:]) #selecionando apenas o ticker, ex : 'VVAR3'
    print(ts1,ts2)
    rates_s1 = mt5.copy_rates_from_pos(ts1, mt5.TIMEFRAME_M1, 0, 1)[0][4]
    rates_s2 = mt5.copy_rates_from_pos(ts1, mt5.TIMEFRAME_M1, 0, 1)[0][4] #baixando ultima cotação 
    index = len(data_par)
    data_par.loc[str(index)] = rates_s1,rates_s2
    data_par_v = data_par.pct_change()[1:]
    data_par_p = data_par[1:]
    
    a = (par_usado[0]+' x '+par_usado[1])
    if not a in lista_pares_usados:
        lista_pares_usados.append(a)
        dic[a] = Pair_trading(data_var=data_par_v, data_preco=data_par_p, par=par_usado, valor_inicial=1000)
        dic[a].opera()
    else:
        dic[a].opera()

BBDC3 BTOW3
ECOR3 GOLL4
CSNA3 MRFG3
QUAL3 USIM5
